In [77]:
# importação das bibliotecas utilizadas
import numpy as np
import os
import pandas as pd

In [78]:
# carregando os filmes indicados ou vencedores do Oscar em um dataframe
oscar_movies = pd.read_csv('/kaggle/input/the-oscar-award/the_oscar_award.csv')

# examinando as características do dataframe
oscar_movies.info()
oscar_movies.describe()

In [79]:
# A única coluna com itens faltantes é a coluna film.
# Vamos explorar as linhas em que faltam esses dados para entender o porquê,
# e em quais categorias isso está ocorrendo

print(oscar_movies[oscar_movies['film'].isnull()]['category'].unique())
oscar_movies[oscar_movies['film'].isnull()].head()

In [80]:
# Alguns desses casos se tratam de prêmios honorários, humanitários e memoriais - 
# ou seja, não relacionados a filmes específicos. Podemos desconsiderar essas linhas

rows_to_drop = oscar_movies[
    (oscar_movies['category'] == 'HONORARY AWARD') | 
    (oscar_movies['category'] == 'SPECIAL AWARD') | 
    (oscar_movies['category'] == 'IRVING G. THALBERG MEMORIAL AWARD') | 
    (oscar_movies['category'] == 'JEAN HERSHOLT HUMANITARIAN AWARD') | 
    (oscar_movies['category'] == 'SPECIAL ACHIEVEMENT AWARD')
].index

oscar_movies = oscar_movies.drop(rows_to_drop)

print(oscar_movies[oscar_movies['film'].isnull()]['category'].unique())
oscar_movies[oscar_movies['film'].isnull()]

In [81]:
# As linhas restantes com dados faltantes na coluna filme são de categorias 
# que precisam necessariamente estar ligadas a um filme.

# Para duas dessas categorias, ligadas a filmes estrangeiros, o nome do filme parece constar
# na coluna 'name'

for row in oscar_movies[
    (oscar_movies['category'] == 'SPECIAL FOREIGN LANGUAGE FILM AWARD') | 
    (oscar_movies['category'] == 'HONORARY FOREIGN LANGUAGE FILM AWARD')
].iterrows():
    print(row[1][4])

# em todos esses casos, com exceção do último, o nome do filme consta na coluna 'name',
# seguido de hífen. Podemos aplicar a mesma regra para todos os casos, e transpor o nome do 
# filme para a coluna film
oscar_movies['film'] = oscar_movies.apply(
    lambda row: row['name'].split('-')[0] if row['category'] in ('SPECIAL FOREIGN LANGUAGE FILM AWARD', 'HONORARY FOREIGN LANGUAGE FILM AWARD') 
    else row['film'],
    axis=1
)

# validando que as alterações foram feitas corretamente
oscar_movies[(oscar_movies['category'] == 'SPECIAL FOREIGN LANGUAGE FILM AWARD') | (oscar_movies['category'] == 'HONORARY FOREIGN LANGUAGE FILM AWARD')]

In [82]:
# Com os dados dessas categorias corrigidos,
# é hora de checar os dados faltantes que ainda restam

print(oscar_movies[oscar_movies['film'].isnull()]['category'].unique())
oscar_movies[oscar_movies['film'].isnull()]